# **Processamento de Linguagem Natural [2024-Q2]**
Prof. Alexandre Donizeti Alves

# **Introdução**
Os indicativos de ativos, especialmente no mercado de commodities, desempenham um papel fundamental para investidores, economistas e empresas. Commodities como petróleo, ouro, e grãos são a base de diversas indústrias, sendo fortemente impactadas por variações de oferta e demanda, políticas econômicas, e eventos globais. A análise desses indicativos permite a antecipação de tendências de preços e movimentos de mercado, ajudando na tomada de decisões estratégicas.

Neste contexto, o uso de Large Language Models (LLMs), tem ganhado destaque. LLMs permitem a análise automatizada de grandes volumes de dados, extraindo insights de notícias e relatórios com rapidez. Além disso, eles podem ser treinados para identificar padrões e fornecer previsões sobre possíveis quedas ou subidas de ativos, como commodities, utilizando dados históricos e variáveis contextuais. A combinação da capacidade de processamento de linguagem natural com algoritmos de machine learning permite gerar relatórios que antecipam oscilações e fornecem recomendações baseadas em informações detalhadas.

Nesse projeto foi realizada uma análise da tendência de preços de commodities, combinando a análise de sentimento de notícias relevantes com a avaliação de séries históricas do ativo. LLM's foram utilizados para auxiliar na extração dessas informações de forma eficiente.


### **Instalando dependências**


In [ ]:
!pip install newsapi-python
!pip install ipywidgets
!pip install langchain_google_genai
!pip install langchain -q U

### **Importando dependências e definindo as APIS KEYS**

In [ ]:
from newsapi import NewsApiClient
import requests
from bs4 import BeautifulSoup
import os
import getpass
import ipywidgets as widgets
import datetime as dt
from datetime import date
import json

os.environ['NEWS_API'] = getpass.getpass()
os.environ["GOOGLE_API_KEY"] = getpass.getpass()


··········
··········


### **Função Para pegar todas as notícias de um determinado Commodity**

Essa função busca artigos sobre uma commodity na NewsAPI e tenta extrair o texto completo de um deles de acordo com os seguintes passos:

- Faz a busca de artigos relacionados à commodity e finanças usando a API NewsAPI.
- Tenta acessar a URL de cada artigo e extrair seu conteúdo com BeautifulSoup.
- Cria um objeto com o título, fonte e texto do artigo.

In [ ]:
def get_all_articles(commodity, sources=None, domains=None, from_param=None, to=None, language='en', sort_by='relevancy', page=1):
    article = ''
    newsapi = NewsApiClient(api_key=os.getenv('NEWS_API'))
    all_articles = newsapi.get_everything(q='financial '+commodity+' commodity price',
                                          sources=sources,
                                          domains=domains,
                                          from_param=from_param,
                                          to=to,
                                          language=language,
                                          sort_by=sort_by,
                                          page=page,
                                          page_size=5)

    articles = all_articles.get('articles')

    if articles:

        for i, article in enumerate(articles, start=1):
            # Tentativa de extrair texto do link do artigo
            try:
                response = requests.get(article['url'])
                soup = BeautifulSoup(response.content, 'html.parser')

                # Extrai o texto do corpo do artigo
                paragraphs = soup.find_all('p')
                article_text = '\n'.join([para.get_text() for para in paragraphs])
                article = Articles(commodity=commodity,title=article['title'], name=article['source']['name'], paragraphs=article_text)
            except Exception as e:
                print(f"   Não foi possível extrair o texto do artigo: {e}")
                article = ''
    else:
        print("Nenhum artigo encontrado.")
    return article

### **Usando classes pydantic**

Validação de Dados: Pydantic permite definir e validar a estrutura dos dados de forma declarativa. Isso é útil para garantir que os dados atendam aos requisitos esperados antes de serem processados ou armazenados.

**Type Hinting**: Utiliza anotações de tipo para validar e converter dados.
Isso ajuda a evitar erros comuns relacionados a tipos de dados e melhora a legibilidade e a manutenção do código.

**Conversão de Dados**: Além de validar, Pydantic pode converter dados para os tipos esperados. Por exemplo, se um campo é esperado como um inteiro, mas o valor é fornecido como uma string, Pydantic pode converter automaticamente para o tipo correto.

**Integração com APIs**: É muito útil para trabalhar com APIs e outras fontes de dados externas. Você pode definir modelos que representam a estrutura dos dados da API e garantir que os dados recebidos estejam corretos.

In [ ]:
from typing import Dict
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field

class Commodity(BaseModel):
    """Informações sobre um commodity."""
    commodity: str = Field(description="Nome do commodity")
    price: str = Field(description="Preço do commodity")
    mv: dict = Field(description="Médias Móveis")

class Status(BaseModel):
    """Informações sobre o status do commodity."""
    status: str = Field(description="Status do commodity")
    commodity: str = Field(description="Nome do commodity")
    price: str = Field(description="Preço do commodity")

class Articles(BaseModel):
    """Informações sobre os artigos."""
    commodity: str = Field(description="Nome do commodity")
    title: str = Field(description="Título do artigo")
    name: str = Field(description="nome do artigo")
    paragraphs: str = Field(description="texto do artigo")

### **Definindo o LLM**

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0, #definir temperatura 0 para o modelo ser mais deterministico
    max_tokens=None,
    timeout=None,
    max_retries=2,
    verbose=True,
)

### **Definindo prompt1**

Esse prompt é responsável por executar a **sumarização** ao extrair a tabela de médias móveis e converter em Json e também capturar o preço do commodity. Usando o *PydanticOutputParser* podemos converter uma saída em uma classe pydantic, no caso a classe foi *Commodity*.

In [ ]:
parser1 = PydanticOutputParser(pydantic_object=Commodity)

prompt1 = PromptTemplate(
     template=(
        "The name of commodity: {commodity} and the price: {price}. You are a skilled expert in HTML data extraction, your task is to analyze the provided HTML table and "
        "convert its content into a structured JSON format. "
        "the price: {price}, capturing the relevant information accurately. \n"
        "{format_instructions} \n"
        "HTML content to analyze:\n"
        "{html}"
    ),
    input_variables=["html", "commodity", "price"],
    partial_variables={"format_instructions": parser1.get_format_instructions()},
)

### **Função Para pegar os indicativos**

É feito um scraping para a **extração de informação** do site *investing.com* para pegar as médias móveis e o preço atual do commodity. O LLM é chamado com o *prompt1* e o invocamos com as respctivas variáveis: *table* (tabela de médias móveis), *commodity* (nome do commodity) e *price* (preço do commodity).

In [ ]:
import requests
from bs4 import BeautifulSoup

def setMAs(commodity):
  request = requests.get('https://br.investing.com/commodities/'+commodity+'-technical')
  soup = BeautifulSoup(request.text, 'html.parser')
  price = soup.find('div', {'class': 'text-5xl/9'}).text
  tables = soup.find_all('table', {'class': 'datatable_table__DE_1_'})
  for i in tables:
      if 'MA5' in i.text:
          table = i
          break
  chain = prompt1 | llm | parser1
  response = chain.invoke({"html": table, "commodity": commodity, "price": price})
  jsonResponse = response.json()
  with open(commodity+".json", "w") as arquivo:
      arquivo.write(jsonResponse)

### **Definindo prompt2**

O *prompt2* passa a instrução para o modelo de análise dos indicativos e dos atigos das notícias por meio da **análise de sentimento**. Foi definido um peso de 15% nos indicativos e 85% nas notícias. Também com a saída foi "paresada" para uma classe pydantic.

In [ ]:
parser2 = PydanticOutputParser(pydantic_object=Status)

prompt2 = PromptTemplate(
     template=(
        "You are a skilled trading expert. Your task is to analyze the provided data on the average exponential and simple moving averages, along with a relevant market article. "
        "Compare the current price with these averages and determine if the commodity is trending up or down based on market rules.\n"
        "Consider the following:\n"
        "If the current price is above both the exponential and simple moving averages, the trend is likely UP.\n"
        "If the current price is below both the exponential and simple moving averages, the trend is likely DOWN.\n"
        "If the current price is above one and below the other, use the majority trend to determine the direction.\n"
        "If the moving average is sloping upwards, it may indicate an uptrend. If it is sloping downwards, it may suggest a downtrend.\n"
        "Additionally, consider the sentiment in the provided article.\n" #considerar o sentimento do artigo
        "If the article suggests positive sentiment (e.g., demand increase or supply constraint), it may confirm an uptrend.\n" #se o article é positivo é possível uma tendência de alta
        "If the article suggests negative sentiment (e.g., surplus supply or weakening demand), it may confirm a downtrend.\n"
        "Please note that the article's sentiment has a weight of 85%, while the technical indicators (moving averages) carry a weight of 15%.\n" #declarando os pesos
        "Your final analysis should reflect these weights when determining the trend direction.\n"
        "Respond with either 'UP' or 'DOWN' followed by the name of the commodity and the price.\n" #instruindo a resposta
        "When the short-term moving average crosses below the long-term moving average, this may suggest a bearish trend.\n"
        "Format Instructions: {format_instructions}\n"
        "Data to analyze: {commodityJson}\n"
        "Article to analyze: {articleJson}"
    ),
    input_variables=["commodityJson","articleJson"],
    partial_variables={"format_instructions": parser2.get_format_instructions()},
)


### **Função para analise final com o LLM**

Nas saídas das médias móveis e nóticias armazenadas em json, a função responsável por analisar os dados acessa esses arquivos *.json* e invoca o llm com prompt2.

In [ ]:
def llm_analisy(commodity):
  with open(commodity+".json", "r") as arquivo:
      MAsJson = arquivo.read()
  with open(commodity+"_articles.json", "r") as arquivo:
      articleJson = arquivo.read()
  chain = prompt2 | llm | parser2
  response = chain.invoke({"commodityJson": MAsJson, "articleJson": articleJson})
  return response.json()

### **Lista dos Commodities**

A lista dos commodities foi retirada conforme a base do investing.com e pode ser acessada aqui: https://br.investing.com/commodities

In [ ]:
commodities = [
    'gold',
    'silver',
    'copper',
    'platinum',
    'brent-oil',
    'crude-oil',
    'natural-gas',
    'heating-oil',
    'us-coffee-c',
    'us-corn',
    'us-wheat',
    'london-sugar',
    'us-cotton-no.2',
    'us-cocoa',
    'oats',
    'palladium',
    'london-gas-oil',
    'aluminum',
    'rough-rice',
    'lumber',
    'feed-cattle',
    'lean-hogs',
    'live-cattle'

]

### **Usando widgets para adicionar um select personalizado**

Usando o widgets para personalizar a experiência do usuário, podemos usar elementos DOM do HTML, o widgets forncece uma ampla variedades de recursos. Abaixo utilizamos o select com a lista dos commodities.

In [ ]:
select = widgets.Select(
    options=commodities,
    value='gold',
    # rows=10,
    description='Commodity:',
    disabled=False
)

display(select)



Select(description='Commodity:', options=('gold', 'silver', 'copper', 'platinum', 'brent-oil', 'crude-oil', 'n…

### **Execução das funções e indicação sobre status do commodity**

In [ ]:
commodity = select.value

articles = get_all_articles(commodity)
with open(commodity+"_articles.json", "w") as arquivo:
    arquivo.write(articles.json())

setMAs(commodity)
info = llm_analisy(commodity)

### **Personalização da saída com widgets**

Criando uma tabela HTML com estilo CSS para a saída do commodity.

In [ ]:
info_json = json.loads(info)
commodity = info_json['commodity']
status = info_json['status']
price = info_json['price']
data_atual = date.today()
data = "{}/{}/{}".format(data_atual.day, data_atual.month,
data_atual.year)

table_html = """
<style>
  table {
    width: 100%;
    border-collapse: separate;
    border-spacing: 0;
    border-radius: 15px;
    overflow: hidden;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
    font-family: Arial, sans-serif;
    text-align: center;
  }
  th, td {
    padding: 12px;
  }
  th {
    background-color: #4CAF50;
    color: white;
  }
  tr:nth-child(even) {
    background-color: #f2f2f2;
  }
  tr:nth-child(odd) {
    background-color: #ffffff;
  }
  td {
    border-bottom: 1px solid #ddd;
  }
  tr:hover {
    background-color: #e0e0e0;
  }
</style>
<table>
  <thead>
    <tr>
      <th>Commodity</th>
      <th>Price (USD)</th>
      <th>Date</th>
      <th>Previsão</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>"""+commodity+"""</td>
      <td>"""+price+"""<//td>
      <td>"""+data+"""<//td>
      <td>"""+status+"""<//td>
    </tr>
  </tbody>
</table>
"""

widgets.HTML(
    value=table_html,
)

HTML(value='\n<style>\n  table {\n    width: 100%;\n    border-collapse: separate;\n    border-spacing: 0;\n  …

### **Referências**

[News Api](https://newsapi.org/)

[Widgets Python](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Basics.html)

[Investing.com](https://www.investing.com/)

[Pydantic](https://docs.pydantic.dev/latest/)